# Which statements do the models get wrong?

In [2]:
import pandas as pd
import json
import numpy as np
from pathlib import Path
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# TRUTH FILE
truth = pd.read_csv("../../Dataset/reddit_data.csv")
#df_binary= pd.read_csv("../w3/qwen/binary_fewshot.json")
# df_multi = 

# Load predictions JSON (list of dicts)
with open("../w3/qwen/binary_fewshot.json") as f:
    preds = json.load(f)
df_binary= pd.DataFrame(preds).drop(columns=['raw_response'])
df_binary
# with open("../w3/qwen/binary_fewshot.json") as f:
#     preds = json.load(f)
# df_binary= pd.DataFrame(preds).drop(columns=['raw_response'])


,id,prompt_type,extracted_label
0,0,fewshot,NOT_IPV
1,1,fewshot,NOT_IPV
2,2,fewshot,NOT_IPV
3,3,fewshot,NOT_IPV
4,4,fewshot,NOT_IPV
...,...,...,...
613,613,fewshot,NOT_IPV
614,614,fewshot,NOT_IPV
615,615,fewshot,NOT_IPV
616,616,fewshot,NOT_IPV


In [4]:
truth

,Unnamed: 0,items,Physical Abuse,Emotional Abuse,Sexual Abuse,Tag,type
0,0,I'm sitting here with a goofy smile and feelin...,False,False,False,False,soft
1,1,It's really boosting my confidence when he say...,False,False,False,False,soft
2,2,I never imagined that someone could make me s...,False,False,False,False,soft
3,3,He motivates me to become the best version of ...,False,False,False,False,soft
4,4,He’s like a best friend that I can also live w...,False,False,False,False,soft
...,...,...,...,...,...,...,...
613,613,He has never said anything negative about the ...,False,False,False,False,soft
614,614,You'd think that in theory online dating would...,False,False,False,False,soft
615,615,I searched for a similar type post because I a...,False,False,False,False,soft
616,616,"So let me explain first, my fiance and me have...",False,False,False,False,soft


In [15]:
wrong_mask = (truth["Tag"] != (df_binary["extracted_label"] == "IPV"))
wrongs = truth.loc[
    wrong_mask,
    ["items", "Physical Abuse", "Emotional Abuse", "Sexual Abuse"]
].copy()
# Add Predictions column
wrongs["Prediction"] = df_binary.loc[wrong_mask, "extracted_label"].values
wrongs

,items,Physical Abuse,Emotional Abuse,Sexual Abuse,Prediction
16,"Sometimes when my partner speaks to me, I feel...",False,True,False,NOT_IPV
18,I know that if she senses frustration directed...,False,True,False,NOT_IPV
20,"As she was yelling, I continued to ask why she...",False,True,False,NOT_IPV
22,She got angry and we blew up again.,False,True,False,NOT_IPV
28,"He says a lot of this was insecurity, validati...",False,True,False,NOT_IPV
...,...,...,...,...,...
390,"Don't come at me saying you're the ""relationsh...",False,False,False,IPV
424,Because calling them sexless virgins is an eas...,False,False,False,IPV
452,You can't be out here getting offended when I ...,False,False,False,IPV
540,"I am in bed now, tearing up after being hung u...",False,False,False,IPV


## What kinds of IPV cases the model is missing?
59/66 wrong predictions are False Negatives (IPV but predicted as NOT_IPV)

In [19]:
false_negatives = wrongs[wrongs["Prediction"] == "NOT_IPV"]
print(len(false_negatives))
false_negatives

59


,items,Physical Abuse,Emotional Abuse,Sexual Abuse,Prediction
16,"Sometimes when my partner speaks to me, I feel...",False,True,False,NOT_IPV
18,I know that if she senses frustration directed...,False,True,False,NOT_IPV
20,"As she was yelling, I continued to ask why she...",False,True,False,NOT_IPV
22,She got angry and we blew up again.,False,True,False,NOT_IPV
28,"He says a lot of this was insecurity, validati...",False,True,False,NOT_IPV
29,The lies are what killed me.,False,True,False,NOT_IPV
37,She has a very short fuse and once she gets an...,False,True,False,NOT_IPV
39,Shes still angry at me about it now hours on.,False,True,False,NOT_IPV
45,"We’ve also fought about a lot of other stuff,...",False,True,False,NOT_IPV
50,He ended up breaking his phone out of anger.,False,True,False,NOT_IPV


## What kinds of non-IPV cases the model thinks are IPV?

In [22]:
false_positives = wrongs[wrongs["Prediction"] == "IPV"]
print(len(false_positives))
for i in false_positives["items"]:
    print(i)

7
If your answer to a direct consequence of your own willing actions is "dismemberment" or "poisoning" of anyone who is inconveniencing you (aka fetus) then you can't be trusted with the reins of your own life, let alone with the (indirect) reins of 4 years of an entire nation's future.What will change my view?Let's start with what will not change it, since I fear most of you will try at least one of these:1.
Some of his rules are;- don’t get married or have kids until you are 30- always wear a condom and make sure you supply the condom- put hot sauce in your condoms after use so if the bitch tries to put your semen in her after sex, she gets quite the burn - never spend more than $40 on a date- never take a woman to get food after the bar because you don’t want her to sober up nor will most women put out with jumbo jack breath- never answer your phones on weekends.
Don't come at me saying you're the "relationship  kind of guy".
Because calling them sexless virgins is an easy way to sh

## Which False Negatives Abuse type the model struggles with most

# Linguistic Pattern Analysis